# Recreating the 'DSOS and SDSOS' Paper's Stable Set Example

The following implementation is meant to recreate with Julia the results obtained by Dr. Ahmadi in: "DSOS and SDSOS Optimization: More Tractable Alternatives to Sum of Squares and Semidefinite Optimization" by Amir Ali Ahmadi and Anirudha Majumdar. See [arXiv:1706.02586](https://arxiv.org/abs/1706.02586).

## Problem Statement
First, we will introduce the problem. The goal of this example (#4.2 in the text) is to identify the largest independent set of a graph $G = (\mathcal{V},\mathcal{E})$ on $n$ nodes.

Recall that an independent set according to Wikipedia is 
> [a set S of vertices such that for every two vertices in S, there is no edge connecting the two](https://en.wikipedia.org/wiki/Independent_set_(graph_theory)).

After using duality and a number of other methods outlined in CITATION NEEDED, this can be posed as the following optimization problem:

$$
\begin{array}{rl}
    \underset{\gamma}{\text{minimize}} & \gamma \\
    \text{subject to} & \gamma \cdot (A+I) - J \in \mathcal{C}_n
\end{array}
$$
where
* $A$ is the adjacency matrix of the graph $G$
* $I$ is the identity matrix
* and $\mathcal{C}_n$ is the set of $n \times n$ copositive matrices

The set of copositive matrices is defined as the following
$$
\mathcal{C}_n = \{ P \in \mathbb{R}^{n \times n} \; | \; x^\top P x \geq 0 \; \forall x \geq 0 \}
$$

## Solution Approach

Such an optimization problem is tricky because it is not easy to optimize over the set of copositive matrices. There is a work around, however, which focuses on the polynomials that can be made with an arbitrary copositive matrix $M \in \mathcal{C}_n$. Consider the following polynomial:
$$ P(x) := (x \circ x)^\top M (x \circ x) = \sum_{i,j=1}^n M_{ij} x_i^2 x_j^2$$
where "$\circ$" indicates the componentwise (Hadamard) product. Now, in this form it is clear that $P(x)$ is nonnegative if and only if the the matrix $M$ is copositive.

If we could verify nonnegativity of a polynomial easily, then this necessary and sufficient condition is all that we would need to test ("Is $P(x) \geq 0 \; \forall x \in \mathbb{R}^n?$") Unfortunately, checking nonnegativity is very difficult. One test that can be done more quickly is the test for a polynomial being Sum of Squares.

Recall that:
> A multivariate polynomial $p:=p(x)$ in $n$ variables and of degree $2d$ is a sum of squares if and only if there exists a symmetric matrix $Q$ (often called the gram matrix) such that
$$
\begin{array}{rcl}
    p(x) & = & z^\top Q z \\
    Q & \succeq & 0
\end{array}
$$
where $z$ is the vector of monomials of degree up to $d$:
$$ z = [1,x_1,x_2,...,x_n,x_1,x_2,...,x_n^d]. $$

In [1]:
#Importing Libraries
import JuMP
import DynamicPolynomials
import SumOfSquares
import LinearAlgebra
import MosekTools #We need an SDP Optimizer

## Example 1: Isocahedron
![Isocahedron Example](https://upload.wikimedia.org/wikipedia/commons/8/83/Icosahedron_graph.svg "ISocahedron Example")



In [2]:
#It's adjacency matrix.
A = [0 1 1 1 1 1 0 0 0 0 0 0;
     1 0 1 1 0 0 0 0 0 1 0 1;
     1 1 0 0 0 1 0 0 0 0 1 1;
     1 1 0 0 1 0 1 0 0 1 0 0;
     1 0 0 1 0 1 1 1 0 0 0 0;
     1 0 1 0 1 0 0 1 0 0 1 0;
     0 0 0 1 1 0 0 1 1 1 0 0;
     0 0 0 0 1 1 1 0 1 0 1 0;
     0 0 0 0 0 0 1 1 0 1 1 1;
     0 1 0 1 0 0 1 0 1 0 0 1;
     0 0 1 0 0 1 0 1 1 0 0 1
     0 1 1 0 0 0 0 0 1 1 1 0]


12×12 Array{Int64,2}:
 0  1  1  1  1  1  0  0  0  0  0  0
 1  0  1  1  0  0  0  0  0  1  0  1
 1  1  0  0  0  1  0  0  0  0  1  1
 1  1  0  0  1  0  1  0  0  1  0  0
 1  0  0  1  0  1  1  1  0  0  0  0
 1  0  1  0  1  0  0  1  0  0  1  0
 0  0  0  1  1  0  0  1  1  1  0  0
 0  0  0  0  1  1  1  0  1  0  1  0
 0  0  0  0  0  0  1  1  0  1  1  1
 0  1  0  1  0  0  1  0  1  0  0  1
 0  0  1  0  0  1  0  1  1  0  0  1
 0  1  1  0  0  0  0  0  1  1  1  0

In [13]:
using DynamicPolynomials
@polyvar x[1:12]

P =  (x.*x)'*A*(x.*x)

#Load the Optimizer
using MosekTools
factory = JuMP.with_optimizer(Mosek.Optimizer, QUIET=true);

r = 2;

using SumOfSquares
model = SOSModel(factory)
@variable(model,gamma)
@objective(model,Min,gamma)
mat_cref2 = @constraint(model, ((x.*x)'*(gamma.*(A+LinearAlgebra.I)-ones(12,12))*(x.*x)).*(x'*x)^r in SDSOSCone())
optimize!(model)
termination_status(model)

InterruptException: InterruptException:

In [12]:
println("gamma = ",JuMP.value(gamma))

gamma = 4.33333333585578
